In [1]:
# Import the necessary packages. 
# Note that you may need to install some packages, e.g., yfinance.
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import urllib.request as ur
from urllib.request import urlopen
import yfinance as yf
import numpy as np

In [2]:
#Create a series of empty lists to store the data,
#which will be used to create a dataframe later.
#These are important categories in a DCF model to calculate unlevered free cash flow.
Revenue_list = []
Operating_Income_list = []
Net_Profit_List = []
Depreciation_list = []
Stock_Based_Compensation_list = []
Change_in_Working_Capital_list = []
Capital_Expenditure_list = []
time_list = []

# We will be using ticker to indicate what company stock you are looking for.
# e.g. ticker = 'AAPL' for Apple Inc.
# ticker is basically a abbreviation for a company stock.
ticker=input("Please enter your ticker")

# Some parameters that will be useful for data acquisition. 

# This is the maximum number of years we can retrieve. 
limit=20
# This is the number of years of data we want to look at.
pastyear=4
# current year
nowyear=2023
# set the time period to annual
# Period is ttm(trailing twelve month) | annual | quarter.
period='annual'
# we are only using the free version, so we can only get annual data. 
# If you want to see ttm/quarter, you need to purchase the premium version.

# We will be using the financialmodelingprep.com API to get the data.
# financialmodelingprep.com is a free API that provides financial data for companies.
# The API key is required to access the data.
# You can get your own API key by registering at https://financialmodelingprep.com/developer
# This is the API key for financialmodelingprep.com
key = '6e9eeec974de71f2e66bb39dfa0f2f71'

def get_income_statement(ticker, limit, key, period):
    """Function to Get the Income Statement."""
    # Set the Income-statement URL for the API
    URL = 'https://financialmodelingprep.com/api/v3/income-statement/'
    try:
        # Get the data from the API
        r = requests.get(
            '{}{}?period={}?limit={}&apikey={}'.format(URL,
                                                       ticker,
                                                       period,
                                                       limit,
                                                       key))
        # Convert the data into a dataframe
        incomeStatement = pd.DataFrame.from_dict(r.json()).transpose()
        # Set the column names to the first row
        incomeStatement.columns = incomeStatement.iloc[0]
        # Drop the first row
        return incomeStatement[1:]
    # If there is an error, print the error
    except requests.exceptions.HTTPError as e:
        print('Requesting Income statement sheet ERROR: ', str(e))

def get_balance_sheet(ticker, limit, key, period):
    """Get the Balance sheet."""
    # Set the Balance Sheet URL for the API
    URL = 'https://financialmodelingprep.com/api/v3/balance-sheet-statement/'
    try:
        # Get the data from the API
        r = requests.get(
            '{}{}?period={}&?limit={}&apikey={}'.format(URL,
                                                        ticker,
                                                        period,
                                                        limit,
                                                        key))
        # Convert the data into a dataframe
        balanceSheet = pd.DataFrame.from_dict(r.json()).transpose()
        # Set the column names to the first row
        balanceSheet.columns = balanceSheet.iloc[0]
        # Drop the first row
        return balanceSheet[1:]
    # If there is an error, print the error
    except requests.exceptions.HTTPError as e:
        print('Requesting Balance sheet statement ERROR: ', str(e))



def get_cash_flow_statement(ticker, limit, key, period):
    """Get the Cash flow statements."""
    # Set the cash flow statements URL for the API
    URL = 'https://financialmodelingprep.com/api/v3/cash-flow-statement/'
    try:
        # Get the data from the API
        r = requests.get(
            '{}{}?period={}&?limit={}&apikey={}'.format(URL,
                                                        ticker,
                                                        period,
                                                        limit,
                                                        key))
        # Convert the data into a dataframe
        cashFlow = pd.DataFrame.from_dict(r.json()).transpose()
        # Set the column names to the first row
        cashFlow.columns = cashFlow.iloc[0]
        # Drop the first row
        return cashFlow[1:]
    # If there is an error, print the error
    except requests.exceptions.HTTPError as e:
        print('Requesting Cash flow statement ERROR: ', str(e))


def get_financial_ratios(ticker, limit, key, period):
    """Get the Financial ratios."""
    """Period is ttm | annual | quarter."""
    # Set the URL for the API
    URL = 'https://financialmodelingprep.com/api/v3/'
    # If the period is ttm, then we need to use different columns than quarter/annual
    if period == "ttm":
        # Get the data from the API
        try:
            r = requests.get(
                '{}/ratios-ttm/{}?{}&apikey={}'.format(URL,
                                                       ticker,
                                                       period,
                                                       key))
            # Convert the data into a dataframe
            fr = pd.DataFrame.from_dict(r.json()).transpose()
            # column names are ticker name + TTM Ratios
            fr.columns = [ticker + " TTM Ratios"]
            return fr
        # If there is an error, print the error
        except requests.exceptions.HTTPError as e:
            print('Requesting Financial ratios ERROR(1): ', str(e))
    # If the period is annual/quarter, then we need to use different columns than ttm
    elif period == "annual" or period == "quarter":
        try:
            # Get the data from the API
            r = requests.get(
                '{}ratios/{}?period={}&?limit={}&apikey={}'.format(URL,
                                                                   ticker,
                                                                   period,
                                                                   limit,
                                                                   key))
            # Convert the data into a dataframe
            fr = pd.DataFrame.from_dict(r.json()).transpose()
            # Set the column names to the second row
            fr.columns = fr.iloc[1]
            # Drop the first two rows
            return fr[2:]
        # If there is an error, print the error
        except requests.exceptions.HTTPError as e:
            print('Requesting Financial ratios ERROR(2): ', str(e))
    # If the period is not ttm/annual/quarter, then we need to print an error
    else:
        print('ERROR: Define the period you want: ttm | annual | quarter')
        return None



In [6]:
# Create a dataframe using the get_income_statement function
df_income_statement=get_income_statement(ticker,limit,key,period)
# Create a dataframe using the get_balance_sheet function
df_balance_sheet=get_balance_sheet(ticker,limit,key,period)
# Create a dataframe using the get_cash_flow_statement function
df_cash_flow_statement=get_cash_flow_statement(ticker,limit,key,period)

In [9]:
df_income_statement

date,2022-09-24,2021-09-25,2020-09-26,2019-09-28,2018-09-29
symbol,AAPL,AAPL,AAPL,AAPL,AAPL
reportedCurrency,USD,USD,USD,USD,USD
cik,0000320193,0000320193,0000320193,0000320193,0000320193
fillingDate,2022-10-28,2021-10-29,2020-10-30,2019-10-31,2018-11-05
acceptedDate,2022-10-27 18:01:14,2021-10-28 18:04:28,2020-10-29 18:06:25,2019-10-30 18:12:36,2018-11-05 08:01:40
calendarYear,2022,2021,2020,2019,2018
period,FY,FY,FY,FY,FY
revenue,394328000000,365817000000,274515000000,260174000000,265595000000
costOfRevenue,223546000000,212981000000,169559000000,161782000000,163756000000
grossProfit,170782000000,152836000000,104956000000,98392000000,101839000000


In [10]:
df_balance_sheet

date,2022-09-24,2021-09-25,2020-09-26,2019-09-28,2018-09-29
symbol,AAPL,AAPL,AAPL,AAPL,AAPL
reportedCurrency,USD,USD,USD,USD,USD
cik,0000320193,0000320193,0000320193,0000320193,0000320193
fillingDate,2022-10-28,2021-10-29,2020-10-30,2019-10-31,2018-11-05
acceptedDate,2022-10-27 18:01:14,2021-10-28 18:04:28,2020-10-29 18:06:25,2019-10-30 18:12:36,2018-11-05 08:01:40
calendarYear,2022,2021,2020,2019,2018
period,FY,FY,FY,FY,FY
cashAndCashEquivalents,23646000000,34940000000,38016000000,48844000000,25913000000
shortTermInvestments,24658000000,27699000000,52927000000,51713000000,40388000000
cashAndShortTermInvestments,48304000000,62639000000,90943000000,100557000000,66301000000


In [11]:
df_cash_flow_statement

date,2022-09-24,2021-09-25,2020-09-26,2019-09-28,2018-09-29
symbol,AAPL,AAPL,AAPL,AAPL,AAPL
reportedCurrency,USD,USD,USD,USD,USD
cik,0000320193,0000320193,0000320193,0000320193,0000320193
fillingDate,2022-10-28,2021-10-29,2020-10-30,2019-10-31,2018-11-05
acceptedDate,2022-10-27 18:01:14,2021-10-28 18:04:28,2020-10-29 18:06:25,2019-10-30 18:12:36,2018-11-05 08:01:40
calendarYear,2022,2021,2020,2019,2018
period,FY,FY,FY,FY,FY
netIncome,99803000000,94680000000,57411000000,55256000000,59531000000
depreciationAndAmortization,11104000000,11284000000,11056000000,12547000000,10903000000
deferredIncomeTax,895000000,-4774000000,-215000000,-340000000,-32590000000
